In [1]:
import pandas as pd
import requests
from tqdm import tqdm
import random
import matplotlib.pyplot as plt

# Useful Links
Implementing data object for neural networks:
https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html

Writing a custom dataset class: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

Writing a GAT Class example: https://github.com/pyg-team/pytorch_geometric/blob/master/examples/gat.py

Dataloaders for Batch Training: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

Splitting datasets for training and testing: https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split

In [2]:
scRNA_data = pd.read_csv('GSE200981_scRNAseq_processed.tsv', sep='\t')
scRNA_data.index = scRNA_data['Gene.names']
scRNA_data = scRNA_data.drop('Gene.names', axis=1)
len(scRNA_data)

26364

In [3]:
#Mapping string to protein names
string_api_url = "https://string-db.org/api"
output_format = "tsv-no-header"
method = "get_string_ids"

params = {

    "identifiers" : "\r".join(list(scRNA_data.index)), # your protein list
    "limit": 1,
    "echo_query": 1,
    "species" : 9606, # species NCBI identifier 
    "caller_identity" : "www.awesome_app.org" # your app name

}

request_url = "/".join([string_api_url, output_format, method])

results = requests.post(request_url, data=params)


protein_2_string = dict()
string_2_protein = dict()

for line in results.text.strip().split("\n"):
    l = line.split("\t")
    protein_identifier, string_identifier = l[0], l[2]
    protein_2_string[protein_identifier] = string_identifier
    string_2_protein[string_identifier] = protein_identifier

In [4]:
scRNA_data = scRNA_data.loc[list(protein_2_string.keys())]
scRNA_data

,V1_T0,V2_T0,V3_T0,V4_T0,V5_T0,V6_T0,V7_T0,V8_T0,V9_T0,V10_T0,...,V247_T7,V248_T7,V249_T7,V250_T7,V251_T7,V252_T7,V253_T7,V254_T7,V255_T7,V256_T7
Gene.names,,,,,,,,,,,,,,,,,,,,,
OR4F5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
OR4F3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
OR4F29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
OR4F16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SAMD11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DAZ1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DAZ3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DAZ2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.nn as nn
from torch_geometric.data.batch import Batch

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
from dataset import EMT_Dataset
dataset = EMT_Dataset(scRNA_data, string_2_protein)

  0%|          | 0/13715404 [00:00<?, ?it/s]

Getting network tensor...


100%|██████████| 13715404/13715404 [00:09<00:00, 1446571.09it/s]


Getting node features tensor...


In [8]:
generator = torch.Generator().manual_seed(42)
train_length = int(0.7*len(dataset))
training_dataset, testing_dataset = random_split(dataset, [train_length, len(dataset)-train_length], generator=generator)

In [9]:
training_dataloader = DataLoader(training_dataset, batch_size=64, shuffle=True)
testing_dataloader = DataLoader(testing_dataset, batch_size=64, shuffle=True)

In [44]:
def train(model, dataloader, epochs, lr = 1e-8, weight_decay = 5e-4):
    torch.cuda.empty_cache()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss().to(device)
    model.train()
    optimizer.zero_grad()
    all_losses = []
    
    for _ in tqdm(range(epochs)):
        model = model.train()
        losses = []
        
        for (graph, node_features, output) in dataloader:
            #print('Graph')
            #print(graph.size())
            optimizer.zero_grad()
            
            #graph, node_features, output = dataset.__getitem__(i)
            graph = Batch(graph.to(device))
            node_features = Batch(node_features.to(device))
            output = Batch(output.to(device))
            #print(graph.size(), node_features.size(), output.size())
            print(node_features)
    
            out = model(node_features, graph)
            #print(out, output)
            loss = criterion(out, output)
            loss.backward()
            optimizer.step()
            losses.append(loss)
            
        all_losses.append(sum(losses)/len(losses))
        
    plt.plot([i for i in range(1, epochs+1)], all_losses)
    plt.xlabel('Number of Epochs')
    plt.ylabel('Loss')
    plt.title('GATConv E/M classification Training') 

In [45]:
from GAT import GAT
model = GAT(1, dataset.num_features(), dataset.num_classes(), 1).to(device)

In [46]:
train(model, training_dataloader, 1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

Batch(batch=[64, 18840, 1])


AttributeError: 'Batch' object has no attribute 'size'

In [ ]:
def test(model, dataloader):
    test_loss = 0
    fp, tp, fn, tn = 0, 0, 0, 0
    criterion = nn.MSELoss().to(device)
    
    for (graph, node_features, output) in dataloader:
        graph = graph.to(device)
        node_features = node_features.to(device)
        output = output.to(device)
        
        out = model(node_features, graph)
        
        loss = criterion(output, label)
        
        test_loss += loss.item() * graph.size(0)
        
    return test_loss/len(dataset)

In [ ]:
test(model, testing_dataloader)

In [13]:
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)

In [14]:
edge_index.size()

torch.Size([2, 4])

In [15]:
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

In [16]:
x.size()

torch.Size([3, 1])